In [1]:
import os
import pandas as pd
import pymysql
import numpy as np

def dev_db_conn():
    """개발 DB 접속 함수"""
    conn = pymysql.connect(
        host = 'db-7ma06.pub-cdb.ntruss.com',
        user = 'redtable',
        password = 'fpemxpdlqmf5491!@#',
        autocommit = True,
        cursorclass= pymysql.cursors.DictCursor)
    return conn


def rt_db_conn():
    """빅데이터센터 DB 접속 함수"""
    conn = pymysql.connect(
        host = '220.95.212.6',
        user = 'red_dev_jiang',
        password = 'redtable211b',
        db = 'redtable_bigdata_center',
        autocommit = True,
        cursorclass= pymysql.cursors.DictCursor)
    conn.query("set character_set_connection=utf8;")
    conn.query("set character_set_server=utf8;")
    conn.query("set character_set_client=utf8;")
    conn.query("set character_set_results=utf8;")
    conn.query("set character_set_database=utf8;")
    return conn


file_path = os.path.join(os.environ["HOMEPATH"], "Downloads/")

# 지방 공항 식당 인기 메뉴 데이터

# RT_LCLT_ARPRT_RSTRNT_POPULAR_MENU_INFO_202407

## https://www.bigdata-culture.kr/bigdata/user/data_market/detail.do?id=bcd33d20-0547-11ec-8e60-2bcdc8456bfb

### 이전 작업했던 기록도 없음. 산출된 데이터를 바탕으로 추측하여 진행.
### 이전 데이터들을 살펴보니 지방 공항 거리가 각자 다름
### 추출기준 : 지방공항 지번주소를 바탕으로 대표메뉴들을 추출하여 진행

In [34]:
def text_extract(input_text):
    """상세주소 제외한 함수"""
    result_text = " ".join(input_text.split()[:-1])
    if result_text.endswith("리"):
        return " ".join(result_text.split()[:-1])
    return result_text


# 공항데이터 추가 (2024-07-09)
data = {
    '0':["1","거점공항","김포국제공항","서울특별시 강서구 방화동 886","서울특별시 강서구 하늘길 38","11540232"],
    '1':["2","거점공항","김해국제공항","부산광역시 강서구 대저2동 2350","부산광역시 강서구 공항진입로 108","13036655"],
    '2':["3","거점공항","제주국제공항","제주특별자치도 제주시 용담이동 2002","제주특별자치도 제주시 공항로 2","11576965"],
    '3':["4","거점공항","대구국제공항","대구광역시 동구 지저동 400-1","대구광역시 동구 공항로 221","11576983"],
    '4':["5","거점공항","청주국제공항","충청북도 청주시 청원구 내수읍 입상리 산5-4","충청북도 청주시 청원구 내수읍 오창대로 980","11576967"],
    '5':["6","거점공항","무안국제공항","전라남도 무안군 망운면 피서리 1065","전라남도 무안군 망운면 공항로 970-260","12301846"],
    '6':["7","일반공항","양양국제공항","강원특별자치도 양양군 손양면 동호리 545","강원특별자치도 양양군 손양면 공항로 201","11576985"],
    '7':["8","일반공항","울산공항","울산광역시 북구 화봉동 522","울산광역시 북구 산업로 1103","11576970"],
    '8':["9","일반공항","여수공항","전라남도 여수시 율촌면 신풍리 979","전라남도 여수시 율촌면 여순로 386","11576974"],
    '9':["10","일반공항","사천공항","경상남도 사천시 사천읍 구암리 1713","경상남도 사천시 사천읍 사천대로 1971","11576975"],
    '10':["11","일반공항","포항경주공항","경상북도 포항시 남구 동해면 도구리 402-1","경상북도 포항시 남구 동해면 일월로 18","11576976"],
    '11':["12","일반공항","군산공항","전북특별자치도 군산시 옥서면 선연리 385","전북특별자치도 군산시 옥서면 산동길 2","11576978"],
    '12':["13","일반공항","원주공항","강원특별자치도 횡성군 횡성읍 곡교리 112","강원특별자치도 횡성군 횡성읍 횡성로 42","11576981"],
    '13':["14","일반공항","광주공항","광주광역시 광산구 신촌동 698-9","광주광역시 광산구 상무대로 420-25","11576966"]
}
df = pd.DataFrame.from_dict(data=data, orient='index', columns=['SN','항목','이름','지번주소','도로명주소','naver_id'])
df["검색주소"] = df["지번주소"].apply(lambda x:text_extract(x))

In [37]:
for i in range(len(df)):
    with dev_db_conn() as conn:
        with conn.cursor() as curs:
            sql = f"""
            SELECT DISTINCT T4.PRODUCT_ID AS 'MENU_ID', %s AS 'ARPRT_NM',
            T3.SRC_ID AS 'RSTRNT_ID', T1.RSTR_NM AS 'RSTRNT_NM', T4.MENU_NM, T4.MENU_PRICE AS 'MENU_PRC',
            GROUP_CONCAT(DISTINCT IF(T5.TYPE_NM = '주재료', T6.TAG_NM, NULL)) AS 'MAIN_INGREDIENT_DC',
            GROUP_CONCAT(DISTINCT IF(T5.TYPE_NM = '조리법', T6.TAG_NM, NULL)) AS 'CKNG_STYLE_DC',
            GROUP_CONCAT(DISTINCT IF(T5.TYPE_NM = '소스', T6.TAG_NM, NULL)) AS 'SOURC_DC',
            GROUP_CONCAT(DISTINCT IF(T5.TYPE_NM = '옵션', T6.TAG_NM, NULL)) AS 'OPTN_DC'
            FROM redtable01.RSTR T1
            INNER JOIN redtable01.RSTR_INDEX T2 ON T1.ID = T2.RSTR_ID
            INNER JOIN redtable01.RSTR_CONN T3 ON T2.RSTR_ID = T3.RSTR_ID
            INNER JOIN (SELECT RSTR_ID, PRODUCT_ID, MENU_NM, MENU_PRICE, ID
                FROM redtable01.MENU
                WHERE PRODUCT_ID IS NOT NULL AND PRODUCT_ID<>'' AND REPRSNT_MENU_YN = 'Y'
                GROUP BY MENU_NM) T4 ON T3.RSTR_ID = T4.RSTR_ID
            LEFT JOIN redtable01.MENU_TAG T5 ON T4.ID = T5.MENU_ID
            LEFT JOIN redtable01.MENU_TAG_MASTER T6 ON T5.MENU_TAG_MASTER_ID = T6.ID
            WHERE T2.IS_USE=1 AND T2.IS_OPEN=1 AND T3.SRC='live_db' AND T1.AREA_NM_1 IS NOT NULL AND T1.AREA_NM_1<>''
            AND T1.RSTR_NM IS NOT NULL AND T1.RSTR_NM<>'' AND T1.RSTR_NM NOT LIKE '%%？%%'
            AND T1.RSTR_LNNO_ADRES LIKE %s
            GROUP BY T4.PRODUCT_ID;
            """
            curs.execute(sql, (df["이름"][i], f'%{df["검색주소"][i]}%'))
            df_result = pd.DataFrame(curs.fetchall())
            
    if i == 0:
        df_final = df_result
    
    else:
        df_final = pd.concat([df_final, df_result])
        
# 결측치 및 불필요한 데이터 정리
df_final = df_final.fillna(np.nan)
df_final.replace({np.nan:None, '\\n':None}, inplace=True)

In [39]:
with rt_db_conn() as conn:
    sql = """
    SELECT TABLE_NAME, COLUMN_NAME, column_comment
    FROM information_schema.`COLUMNS`
    WHERE TABLE_NAME = 'RT_LCLT_ARPRT_RSTRNT_POPULAR_MENU_INFO_202407'
    """
    columns = pd.read_sql(sql, conn)
    
# 컬럼명 가져오기
db_cols = ','.join(columns['COLUMN_NAME'])

In [43]:
with rt_db_conn() as conn:
    with conn.cursor() as curs:
        sql = f"""
        INSERT IGNORE INTO redtable_bigdata_center.RT_LCLT_ARPRT_RSTRNT_POPULAR_MENU_INFO_202407({db_cols})
        VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
        """
        val = df_final.apply(lambda row:(row["MENU_ID"], row["ARPRT_NM"], row["RSTRNT_ID"], row["RSTRNT_NM"],
                                  row["MENU_NM"], row["MENU_PRC"], row["MAIN_INGREDIENT_DC"], row["CKNG_STYLE_DC"],
                                  row["SOURC_DC"], row["OPTN_DC"]), axis=1).tolist()
        curs.executemany(sql, val)